In [6]:
import pandas as pd
import os
from dotenv import load_dotenv
from zhipuai import ZhipuAI
load_dotenv()
client = ZhipuAI(api_key=os.getenv("ZHIPUAI_API_KEY"))

In [2]:
df_chunks = pd.read_csv('chunks.csv', sep="|", index_col=0)
df_chunks

,text,source,page
0,李产业发展现状\n一、中国李产业现状\n(一)栽培历史\n我国李栽培历史久远，原始居民就有采...,./李高效生产及绿色防控技术.pdf,0
1,李高效生产及绿色防控技术\n考证，“藉栽”指自根繁殖，即包括扦插、压条、分株等无性繁殖方\n...,./李高效生产及绿色防控技术.pdf,1
2,"第一章李产业发展现状\n年积温在4500~8000℃,最冷月平均气温0~15℃,是热带与温带...",./李高效生产及绿色防控技术.pdf,2
3,李高效生产及绿色防控技术\n(3)资源品种。本区主要栽培种为中国李，部分为欧洲李、杏李\n和...,./李高效生产及绿色防控技术.pdf,3
4,第一章李产业发展现状\n2绥棱红(北方1号)，它的母本和父本分别来自于小黄李和福摩萨，\n由...,./李高效生产及绿色防控技术.pdf,4
...,...,...,...
145,第五章李贮运保鲜与加工\n(5)预煮。预煮前用1.5%的食盐水护色，最后用清水淘洗1次。\n...,./李高效生产及绿色防控技术.pdf,145
146,李高效生产及绿色防控技术\n2.主要加工设施设备\n(1)清洗设备。鼓泡清洗机、毛刷清洗机、...,./李高效生产及绿色防控技术.pdf,146
147,第五章李贮运保鲜与加工\n4.产品质量要求\n(1)感官指标。具有该品种李加工后应有的正常颜...,./李高效生产及绿色防控技术.pdf,147
148,李高效生产及绿色防控技术\n(3)清洗。采用果蔬专用清洗设备清洗表面灰尘、腐叶、虫卵等\n污...,./李高效生产及绿色防控技术.pdf,148


In [19]:
SYSTEM_PROMPT_GENQA = """
你是一个问答数据集构建专家，用户会给你一段文本，你的任务是准确地从这段文本里提取出问题和答案，你可以按下面步骤进行：
1. 可以对文本里的概念、时间、地点等进行提问。
2. 同一个问题可能有多种不同的答案，尽可能增加问题的多样性。
3. 将提取出的问题和答案按照如下格式示例输出，只需要输出结果，不需要解释思路。
格式示例：
   法国的首都是哪儿|巴黎
   2020年东京奥运会在哪一年举行|2021年
   蓝莓又名什么|越橘
"""
def genqa(text: str):
  res = client.chat.completions.create(
    model="glm-3-turbo",
    messages=[
      {"role": "system", "content": SYSTEM_PROMPT_GENQA},
      {"role": "user", "content": text},
    ]
  )
  return res.choices[0].message

In [20]:
from tqdm import tqdm

with open("raw/qa-0.csv", "w") as f:
  for i, row in tqdm(df_chunks.iterrows()):
    if i >= 0:
      content = genqa(row["text"]).content
      lines = content.split("\n")
      for line in lines:
        f.write(f"{line}|{i}\n")
      f.flush()

150it [18:03,  7.23s/it]


In [49]:
with open("raw/qa-0-149.csv") as r:
    with open("raw/qa-bad.csv", "w") as w_bad:
        with open("raw/qa.csv", "w") as w_good:
            for line in r:
                cells = line.split("|")
                match len(cells):
                    case 3:
                        w_good.write(f"{cells[0]}|{cells[1]}|{cells[2]}")
                    case _:
                        if cells[0]:
                            w_bad.write(f"{line}")

In [54]:
pairs: list[tuple[str, int]] = []
with open("raw/qa-bad.csv") as r:
    for line in r:
        cells = line.split("|")
        if cells[0].startswith("问题") or cells[0].startswith("答案"):
            pairs.append((cells[0][4:], int(cells[1].strip())))
with open("raw/qa.csv", "a") as w:
    for i in range(0, len(pairs), 2):
        w.write(f"{pairs[i][0]}|{pairs[i+1][0]}|{pairs[i][1]}\n")
        

In [69]:
df_qa = pd.read_csv("raw/qa.csv", sep="|", names=["q", "a", "d"]).sort_values("d").reset_index(drop=True)
df_qa.to_csv("preprocessed/qa.csv", sep="|")
df_qa

,q,a,d
0,请问中国李产业的历史可以追溯到什么时候？,远在5000多年前,0
1,中国最早关于李的文献记载出现在哪部古籍中？,《诗经》,0
2,《广志》记载汉代上林苑收集了多少个李品种？,15个,0
3,《尔雅》中记录了哪些李品种？,椄虑李、无实李、赤李,0
4,《山海经》中提到的灵山有哪些果树？,桃、李、梅、杏,0
...,...,...,...
838,李高效生产及绿色防控技术的具体步骤是怎样的？,李高效生产及绿色防控技术的步骤包括清洗、破碎打浆、加热浓缩、洗瓶、装罐及密封和杀菌冷却。,148
839,在李果酱的生产过程中，原浆是如何处理的？,在李果酱的生产过程中，原浆是通过破碎打浆机将李打成原浆，并在打浆过程中添加0.2%~0.5%...,148
840,李果酱在装罐封口时的温度要求是多少？,李果酱在装罐封口时的温度要求在80℃以上，以防止微生物污染。,148
841,李果酱的产品质量要求有哪些？,李果酱的产品质量要求包括感官指标、理化指标、污染物限量和微生物限量。感官指标要求具有该产品应...,148


In [91]:
SYSTEM_PROMPT_GENEVAL = """
你是一个将问答题转换成单项选择题的专家，用户会给你问题和答案，你的任务是准确地将该问题和答案转换成问题、选项、正确选项，你可以按下面步骤进行：
1. 根据给的问题，生成三个迷惑性很强的错误选项，并将给的答案设置为正确选项，不要改变给的答案原本的内容。
2. 只能有一个正确选项和三个错误选项，要求四个选项的顺序随机。
3. 转换结果按照如下示例输出，只需要输出结果，不需要解释思路。
示例：
   问题：法国的首都是哪儿？
   选项：A.华盛顿 B.东京 C.巴黎 D.伦敦
   正确选项：C
"""
def geneval(text: str):
  res = client.chat.completions.create(
    model="glm-3-turbo",
    messages=[
      {"role": "system", "content": SYSTEM_PROMPT_GENEVAL},
      {"role": "user", "content": text},
    ]
  )
  return res.choices[0].message

In [90]:
geneval(f"问题：{df_qa.iloc[0]["q"]}\n答案：{df_qa.iloc[0]["a"]}")

CompletionMessage(content='问题：中国李产业的历史可以追溯到什么时候？\nA. 新石器时代晚期\nB. 春秋战国时期\nC. 公元10世纪\nD. 5000多年前\n正确选项：D', role='assistant', tool_calls=None)

In [ ]:
from tqdm import tqdm

with open("preprocessed/eval-478.csv", "w") as w:
  for i, row in tqdm(df_qa.iterrows()):
    if i >= 478:
      content = geneval(f"问题：{row["q"]}\n答案：{row["a"]}").content
      w.write(f"{content}\n\n")
      w.flush()